In [1]:
# join identifiers
# change "pathology" column values to 0, 1 and 2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_train = pd.read_csv('/content/drive/My Drive/machine-learning-project/data/mass_case_description_train_set.csv')
df_train.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/machine-learning-project/data/mass_case_description_train_set.csv'

In [ ]:
# join the patient_id, left or right breast, image view, and abnormality id columns into one with an underscore separating each element
df_train['identifier'] = df_train['patient_id'].astype(str) + '_' + df_train['left or right breast'].astype(str) + '_' + df_train['image view'].astype(str) + '_' + df_train['abnormality id'].astype(str)
df_train.head()

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,mass shape,mass margins,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path,identifier
0,P_00001,3,LEFT,CC,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,Mass-Training_P_00001_LEFT_CC/1.3.6.1.4.1.9590...,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...,P_00001_LEFT_CC_1
1,P_00001,3,LEFT,MLO,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,Mass-Training_P_00001_LEFT_MLO/1.3.6.1.4.1.959...,Mass-Training_P_00001_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00001_LEFT_MLO_1/1.3.6.1.4.1.9...,P_00001_LEFT_MLO_1
2,P_00004,3,LEFT,CC,1,mass,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,Mass-Training_P_00004_LEFT_CC/1.3.6.1.4.1.9590...,Mass-Training_P_00004_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00004_LEFT_CC_1/1.3.6.1.4.1.95...,P_00004_LEFT_CC_1
3,P_00004,3,LEFT,MLO,1,mass,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,Mass-Training_P_00004_LEFT_MLO/1.3.6.1.4.1.959...,Mass-Training_P_00004_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00004_LEFT_MLO_1/1.3.6.1.4.1.9...,P_00004_LEFT_MLO_1
4,P_00004,3,RIGHT,MLO,1,mass,OVAL,CIRCUMSCRIBED,4,BENIGN,5,Mass-Training_P_00004_RIGHT_MLO/1.3.6.1.4.1.95...,Mass-Training_P_00004_RIGHT_MLO_1/1.3.6.1.4.1....,Mass-Training_P_00004_RIGHT_MLO_1/1.3.6.1.4.1....,P_00004_RIGHT_MLO_1


In [ ]:
# drop patient_id, left or right breast, image view, and abnormality id, abnormality type, image file path, cropped image file path, ROI mask file path columns
df_train = df_train.drop(['patient_id', 'left or right breast', 'image view', 'abnormality id', 'abnormality type', 'image file path', 'cropped image file path', 'ROI mask file path'], axis=1)
df_train.head()

,breast_density,mass shape,mass margins,assessment,pathology,subtlety,identifier
0,3,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,P_00001_LEFT_CC_1
1,3,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,P_00001_LEFT_MLO_1
2,3,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,P_00004_LEFT_CC_1
3,3,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,P_00004_LEFT_MLO_1
4,3,OVAL,CIRCUMSCRIBED,4,BENIGN,5,P_00004_RIGHT_MLO_1


In [ ]:
# reorder the columns: identifier, mass shape, mass margins, breast_density, assessment, subtlety, pathology
df_train = df_train[['identifier', 'mass shape', 'mass margins', 'breast_density', 'assessment', 'subtlety', 'pathology']]
df_train.head()

,identifier,mass shape,mass margins,breast_density,assessment,subtlety,pathology
0,P_00001_LEFT_CC_1,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,3,4,4,MALIGNANT
1,P_00001_LEFT_MLO_1,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,3,4,4,MALIGNANT
2,P_00004_LEFT_CC_1,ARCHITECTURAL_DISTORTION,ILL_DEFINED,3,4,3,BENIGN
3,P_00004_LEFT_MLO_1,ARCHITECTURAL_DISTORTION,ILL_DEFINED,3,4,3,BENIGN
4,P_00004_RIGHT_MLO_1,OVAL,CIRCUMSCRIBED,3,4,5,BENIGN


In [ ]:
# in the pathology column, reclassify "BENIGN" as 0, "BENIGN_WITHOUT_CALLBACK" as 1, and "MALIGNANT" as 2
df_train['pathology'] = df_train['pathology'].replace({'BENIGN': 0, 'BENIGN_WITHOUT_CALLBACK': 1, 'MALIGNANT': 2})
df_train.head()

<ipython-input-7-210a429f0e58>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['pathology'] = df_train['pathology'].replace({'BENIGN': 0, 'BENIGN_WITHOUT_CALLBACK': 1, 'MALIGNANT': 2})


,identifier,mass shape,mass margins,breast_density,assessment,subtlety,pathology
0,P_00001_LEFT_CC_1,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,3,4,4,2
1,P_00001_LEFT_MLO_1,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,3,4,4,2
2,P_00004_LEFT_CC_1,ARCHITECTURAL_DISTORTION,ILL_DEFINED,3,4,3,0
3,P_00004_LEFT_MLO_1,ARCHITECTURAL_DISTORTION,ILL_DEFINED,3,4,3,0
4,P_00004_RIGHT_MLO_1,OVAL,CIRCUMSCRIBED,3,4,5,0


In [ ]:
# read in testing data
df_test = pd.read_csv('/content/drive/My Drive/machine-learning-project/data/mass_case_description_test_set.csv')
df_test.head()

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,mass shape,mass margins,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00016,4,LEFT,CC,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,5,Mass-Test_P_00016_LEFT_CC/1.3.6.1.4.1.9590.100...,Mass-Test_P_00016_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Mass-Test_P_00016_LEFT_CC_1/1.3.6.1.4.1.9590.1...
1,P_00016,4,LEFT,MLO,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,5,Mass-Test_P_00016_LEFT_MLO/1.3.6.1.4.1.9590.10...,Mass-Test_P_00016_LEFT_MLO_1/1.3.6.1.4.1.9590....,Mass-Test_P_00016_LEFT_MLO_1/1.3.6.1.4.1.9590....
2,P_00017,2,LEFT,CC,1,mass,ROUND,CIRCUMSCRIBED,4,MALIGNANT,4,Mass-Test_P_00017_LEFT_CC/1.3.6.1.4.1.9590.100...,Mass-Test_P_00017_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Mass-Test_P_00017_LEFT_CC_1/1.3.6.1.4.1.9590.1...
3,P_00017,2,LEFT,MLO,1,mass,ROUND,ILL_DEFINED,4,MALIGNANT,4,Mass-Test_P_00017_LEFT_MLO/1.3.6.1.4.1.9590.10...,Mass-Test_P_00017_LEFT_MLO_1/1.3.6.1.4.1.9590....,Mass-Test_P_00017_LEFT_MLO_1/1.3.6.1.4.1.9590....
4,P_00032,3,RIGHT,CC,1,mass,ROUND,OBSCURED,0,BENIGN,2,Mass-Test_P_00032_RIGHT_CC/1.3.6.1.4.1.9590.10...,Mass-Test_P_00032_RIGHT_CC_1/1.3.6.1.4.1.9590....,Mass-Test_P_00032_RIGHT_CC_1/1.3.6.1.4.1.9590....


In [ ]:
# join the patient_id, left or right breast, image view, and abnormality id columns into one with an underscore separating each element
df_test['identifier'] = df_test['patient_id'].astype(str) + '_' + df_test['left or right breast'].astype(str) + '_' + df_test['image view'].astype(str) + '_' + df_test['abnormality id'].astype(str)
df_test.head()

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,mass shape,mass margins,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path,identifier
0,P_00016,4,LEFT,CC,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,5,Mass-Test_P_00016_LEFT_CC/1.3.6.1.4.1.9590.100...,Mass-Test_P_00016_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Mass-Test_P_00016_LEFT_CC_1/1.3.6.1.4.1.9590.1...,P_00016_LEFT_CC_1
1,P_00016,4,LEFT,MLO,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,5,Mass-Test_P_00016_LEFT_MLO/1.3.6.1.4.1.9590.10...,Mass-Test_P_00016_LEFT_MLO_1/1.3.6.1.4.1.9590....,Mass-Test_P_00016_LEFT_MLO_1/1.3.6.1.4.1.9590....,P_00016_LEFT_MLO_1
2,P_00017,2,LEFT,CC,1,mass,ROUND,CIRCUMSCRIBED,4,MALIGNANT,4,Mass-Test_P_00017_LEFT_CC/1.3.6.1.4.1.9590.100...,Mass-Test_P_00017_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Mass-Test_P_00017_LEFT_CC_1/1.3.6.1.4.1.9590.1...,P_00017_LEFT_CC_1
3,P_00017,2,LEFT,MLO,1,mass,ROUND,ILL_DEFINED,4,MALIGNANT,4,Mass-Test_P_00017_LEFT_MLO/1.3.6.1.4.1.9590.10...,Mass-Test_P_00017_LEFT_MLO_1/1.3.6.1.4.1.9590....,Mass-Test_P_00017_LEFT_MLO_1/1.3.6.1.4.1.9590....,P_00017_LEFT_MLO_1
4,P_00032,3,RIGHT,CC,1,mass,ROUND,OBSCURED,0,BENIGN,2,Mass-Test_P_00032_RIGHT_CC/1.3.6.1.4.1.9590.10...,Mass-Test_P_00032_RIGHT_CC_1/1.3.6.1.4.1.9590....,Mass-Test_P_00032_RIGHT_CC_1/1.3.6.1.4.1.9590....,P_00032_RIGHT_CC_1


In [ ]:
# drop patient_id, left or right breast, image view, and abnormality id, abnormality type, image file path, cropped image file path, ROI mask file path columns
df_test = df_test.drop(['patient_id', 'left or right breast', 'image view', 'abnormality id', 'abnormality type', 'image file path', 'cropped image file path', 'ROI mask file path'], axis=1)
df_test.head()

,breast_density,mass shape,mass margins,assessment,pathology,subtlety,identifier
0,4,IRREGULAR,SPICULATED,5,MALIGNANT,5,P_00016_LEFT_CC_1
1,4,IRREGULAR,SPICULATED,5,MALIGNANT,5,P_00016_LEFT_MLO_1
2,2,ROUND,CIRCUMSCRIBED,4,MALIGNANT,4,P_00017_LEFT_CC_1
3,2,ROUND,ILL_DEFINED,4,MALIGNANT,4,P_00017_LEFT_MLO_1
4,3,ROUND,OBSCURED,0,BENIGN,2,P_00032_RIGHT_CC_1


In [ ]:
# reorder the columns: identifier, mass shape, mass margins, breast_density, assessment, subtlety, pathology
df_test = df_test[['identifier', 'mass shape', 'mass margins', 'breast_density', 'assessment', 'subtlety', 'pathology']]
df_test.head()

,identifier,mass shape,mass margins,breast_density,assessment,subtlety,pathology
0,P_00016_LEFT_CC_1,IRREGULAR,SPICULATED,4,5,5,MALIGNANT
1,P_00016_LEFT_MLO_1,IRREGULAR,SPICULATED,4,5,5,MALIGNANT
2,P_00017_LEFT_CC_1,ROUND,CIRCUMSCRIBED,2,4,4,MALIGNANT
3,P_00017_LEFT_MLO_1,ROUND,ILL_DEFINED,2,4,4,MALIGNANT
4,P_00032_RIGHT_CC_1,ROUND,OBSCURED,3,0,2,BENIGN


In [ ]:
# in the pathology column, reclassify "BENIGN" as 0, "BENIGN_WITHOUT_CALLBACK" as 1, and "MALIGNANT" as 2
df_test['pathology'] = df_test['pathology'].replace({'BENIGN': 0, 'BENIGN_WITHOUT_CALLBACK': 1, 'MALIGNANT': 2})
df_test.head()

<ipython-input-12-1939189058bf>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test['pathology'] = df_test['pathology'].replace({'BENIGN': 0, 'BENIGN_WITHOUT_CALLBACK': 1, 'MALIGNANT': 2})


,identifier,mass shape,mass margins,breast_density,assessment,subtlety,pathology
0,P_00016_LEFT_CC_1,IRREGULAR,SPICULATED,4,5,5,2
1,P_00016_LEFT_MLO_1,IRREGULAR,SPICULATED,4,5,5,2
2,P_00017_LEFT_CC_1,ROUND,CIRCUMSCRIBED,2,4,4,2
3,P_00017_LEFT_MLO_1,ROUND,ILL_DEFINED,2,4,4,2
4,P_00032_RIGHT_CC_1,ROUND,OBSCURED,3,0,2,0


In [ ]:
# export df_train and df_test to the data folder as csvs
df_train.to_csv('/content/drive/My Drive/machine-learning-project/data/df_train.csv', index=False)
df_test.to_csv('/content/drive/My Drive/machine-learning-project/data/df_test.csv', index=False)

In [ ]:
# Applying one-hot encoding (dummies) to the specified columns
df_train_encoded = pd.get_dummies(df_train, columns=['identifier', 'mass shape', 'mass margins']).astype(int)

# Displaying the transformed dataframe
df_train_encoded.head()

,breast_density,assessment,subtlety,pathology,identifier_P_00001_LEFT_CC_1,identifier_P_00001_LEFT_MLO_1,identifier_P_00004_LEFT_CC_1,identifier_P_00004_LEFT_MLO_1,identifier_P_00004_RIGHT_MLO_1,identifier_P_00009_RIGHT_CC_1,...,mass margins_ILL_DEFINED-SPICULATED,mass margins_MICROLOBULATED,mass margins_MICROLOBULATED-ILL_DEFINED,mass margins_MICROLOBULATED-ILL_DEFINED-SPICULATED,mass margins_MICROLOBULATED-SPICULATED,mass margins_OBSCURED,mass margins_OBSCURED-ILL_DEFINED,mass margins_OBSCURED-ILL_DEFINED-SPICULATED,mass margins_OBSCURED-SPICULATED,mass margins_SPICULATED
0,3,4,4,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3,4,4,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3,4,3,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,4,3,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,4,5,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# designate target variables (pathology column) as y and feature variables (all other columns) as X
y_train = df_train['pathology']
X_train = df_train.drop(['pathology'], axis=1)

# do the same for testing dataset
y_test = df_test['pathology']
X_test = df_test.drop(['pathology'], axis=1)